In [1]:
import mysql.connector
from faker import Faker
import random
from datetime import datetime

In [ ]:
# Conectar ao banco de dados
conn = mysql.connector.connect(
    host='localhost',       # nome por padrão se o seu tiver outro, tem que trocar
    user='root',            # nome por padrão se o seu tiver outro, tem que trocar
    password='',    	    # sua senha 
    database='gestaorh'     # nome do seu banco de dados
)
cursor = conn.cursor()

fake = Faker('pt_BR')

In [3]:
# Inserir Departamentos
departamento = ['Recursos Humanos', 'Financeiro', 'TI', 'Marketing', 'Vendas']
for nome in departamento:
    descricao = f'Departamento de {nome}'
    cursor.execute("INSERT INTO Departamento (nome, descricao) VALUES (%s, %s)", (nome, descricao))


In [4]:
# Inserir Cargos
cargo = [
    ('Analista de RH', 3500.00),
    ('Assistente Financeiro', 2800.00),
    ('Desenvolvedor Back-end', 5000.00),
    ('Analista de Marketing', 4000.00),
    ('Executivo de Vendas', 4500.00)
]
for nome, salarioBase in cargo:
    descricao = f'Responsável por {nome}'
    cursor.execute("INSERT INTO Cargo (nome, descricao, salarioBase) VALUES (%s, %s, %s)", (nome, descricao, salarioBase))

conn.commit()

In [5]:
# Inserir Funcionários
for _ in range(100):
    nome = fake.name()
    cpf = fake.cpf().replace('.', '').replace('-', '')
    dataNasc = fake.date_of_birth(minimum_age=20, maximum_age=50)
    sexo = random.choice(['M', 'F'])
    email = fake.email()
    telefone = fake.phone_number().replace('(','').replace(')', '').replace('+', '').replace( ' ', '').replace('55', '')
    dataAdmissao = fake.date_between(start_date='-10y', end_date='today')
    idCargo = random.randint(1, len(cargo))
    idDepart = random.randint(1, len(departamento))
    salarioAtual = round(random.uniform(2500, 7000), 2)

    cursor.execute("""
        INSERT INTO Funcionarios 
        (nome, cpf, dataNasc, sexo, email, telefone, dataAdmissao, idCargo, idDepart, salarioAtual) 
        VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
    """, (nome, cpf, dataNasc, sexo, email, telefone, dataAdmissao, idCargo, idDepart, salarioAtual))

conn.commit()

In [6]:
# Obter IDs de funcionários
cursor.execute("SELECT idFuncionario FROM Funcionarios")
func_ids = [row[0] for row in cursor.fetchall()]

# Inserir Histórico Salarial
for func_id in func_ids:
    num_historico = random.randint(1, 3)
    salario = round(random.uniform(2500, 7000), 2)
    for _ in range(num_historico):
        dataAlteracao = fake.date_between(start_date='-5y', end_date='today')
        motivo = random.choice(['Promoção', 'Reajuste', 'Admissão'])
        cursor.execute("""
            INSERT INTO historicosalarial (idFuncionario, salario, dataAlteracao, motivo) 
            VALUES (%s, %s, %s, %s)
        """, (func_id, salario, dataAlteracao, motivo))
        salario += round(random.uniform(100, 500), 2)

conn.commit()

In [7]:
# Inserir Avaliações
for func_id in func_ids:
    num_avaliacoes = random.randint(1, 3)
    for _ in range(num_avaliacoes):
        dataAvaliacao = fake.date_between(start_date='-3y', end_date='today')
        nota = round(random.uniform(3.0, 5.0), 2)
        comentario = fake.sentence()
        avaliador = fake.name()
        cursor.execute("""
            INSERT INTO Avaliacao (idFuncionario, dataAvaliacao, nota, comentario, avaliador) 
            VALUES (%s, %s, %s, %s, %s)
        """, (func_id, dataAvaliacao, nota, comentario, avaliador))

conn.commit()

cursor.close()
conn.close()

print("Dados inseridos com sucesso!")

Dados inseridos com sucesso!
